# Export Live Database
Export data from the running database. For recipient, export `recipient` and original data in `transaction`. Merge them and remove duplicates.

In [1]:
import mysql.connector
import pandas as pd

## Connection

In [2]:
db_connection = mysql.connector.connect(
    host="localhost",
    user="root",
    passwd="",
    database = "pharmagelder"
)

## Load data

In [3]:
#Load recipient and transaction
df_recipient = pd.read_sql("""SELECT
    rec_id as id,
    rec_name as name,
    rec_address as address,
    rec_location as location,
    rec_type as type
  FROM recipient""", con=db_connection)

df_transaction = pd.read_sql("""SELECT
    tra_fk_recipient as id,
    tra_name_original as name,
    tra_location_original as location,
    tra_address_original as address,
    rec_type as type
  FROM transaction
  LEFT JOIN recipient ON tra_fk_recipient = rec_id
  GROUP BY tra_fk_recipient""", con=db_connection)

#Merge datasets
df_recipient = pd.concat([df_recipient, df_transaction], sort=False)

#Drop duplicates
df_recipient.drop_duplicates(inplace = True)

df_recipient = df_recipient.sort_values(by='id')

#Load Pharma
df_pharma = pd.read_sql("SELECT * FROM pharma", con=db_connection)

In [4]:
#Export
df_recipient.to_csv('../../data/0. live data/0_recipient.csv', index=False)
df_pharma.to_csv('../../data/0. live data/0_pharma.csv', index=False)